In [29]:
%reload_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import matplotlib.animation as animation
from scipy.stats import binom
import matplotlib as mpl
from sklearn import datasets
import itertools

from utils import *

In [43]:
X_0 = np.random.rand(50,2) + 1
X_1 = np.random.rand(50,2) - 1
X = np.concatenate([X_0, X_1])

x_min, x_max = min(X[:,0]), max(X[:,0])
y_min, y_max = min(X[:,1]), max(X[:,1])

In [31]:
W, D_sqinv = compute_G0(X)
L = compute_laplacian(W, D_sqinv)
A = compute_AM_normalised(L, D_sqinv, 3) #3 peut être faible
W, g = compute_Ge(A)

In [32]:
g.compute_A()
g.kruskal_algo()
mat = g.compute_mat()

In [ ]:
def update_deltas(deltas, eps):
    

In [58]:
K = 100
deltas = 0.999 * np.ones(K)

X_H0 = np.random.uniform(size=(K,2)) * np.array([x_max - x_min, y_max - y_min]) + np.array([x_min, y_min])

